In [1]:
import transformers
import sqlite3
import jsonlines
import random
from tqdm import tqdm

# Show the MSMARCO data format

In [2]:
!ls /home/ryparmar/MSMARCO-Passage-Ranking/

collection.tar.gz  queries.dev.tsv   queries.train.tsv
qrels.dev.tsv	   queries.eval.tsv  triples.train.small.tar.gz
qrels.train.tsv    queries.tar.gz    triples.train.small.tsv


In [3]:
!zcat /home/ryparmar/MSMARCO-Passage-Ranking/collection.tar.gz | head -n 2

collection.tsv                                                                                      0000777 0001750 0001750 26636750554 13400073633 015704  0                                                                                                    ustar   spacemanidol                    spacemanidol                                                                                                                                                                                                           0	The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.
1	The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on his

In [4]:
!zcat /home/ryparmar/MSMARCO-Passage-Ranking/triples.train.small.tar.gz | head -n 2

triples.train.small.tsv                                                                             0000770 �    z�4001001 �      � �13321546425 016554  0                                                                                                    ustar   bmitra                          REDMOND+Group(513)                                                                                                                                                                                                     is a little caffeine ok during pregnancy	We donât know a lot about the effects of caffeine during pregnancy on you and your baby. So itâs best to limit the amount you get each day. If youâre pregnant, limit caffeine to 200 milligrams each day. This is about the amount in 1Â½ 8-ounce cups of coffee or one 12-ounce cup of coffee.	It is generally safe for pregnant women to eat chocolate because studies have shown to prove certain benefits of eating chocolate during pregnancy. Howe

In [5]:
!zcat /home/ryparmar/MSMARCO-Passage-Ranking/queries.tar.gz | head -n 2

queries.dev.tsv                                                                                     0000664 0001750 0001750 00020762502 13372646016 014016  0                                                                                                    ustar   erasmus                         erasmus                                                                                                                                                                                                                1048578	cost of endless pools/swim spa
1048579	what is pcnt

gzip: stdout: Broken pipe


# Convert FEVER data into MSMARCO format

query \t positive_passage \t negative_passage

or 

query_id \t pos_pass_id \t neg_pass_id

In [42]:
feverdb = '/mnt/data/factcheck/fever/data-cs/fever/fever.db'
# train = '/mnt/data/factcheck/fever/data-en-latest/fever-data/train.jsonl'
# dev = '/mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl'

train = '/home/ryparmar/fever-cs-deepl/train.jsonl'
dev = '/home/ryparmar/fever-cs-deepl/dev.jsonl'
test = '/home/ryparmar/fever-cs-deepl/test.jsonl'

ROOT = '/home/ryparmar/fever-cs-deepl/'

In [43]:
conn = sqlite3.connect(feverdb)  
c = conn.cursor()
c.execute("SELECT * FROM documents")
documents = {r[0]: r[1] for r in c}

In [44]:
num_documents = len(documents)
num_documents

451629

### Create collection file

In [45]:
collection = '/mnt/data/factcheck/fever/data-cs/fever/fever.tsv'
# with open(collection, 'w') as fw:
#     for i, (d, dv) in enumerate(documents.items()):
#         fw.write(f"{i}\t{dv}\n")
        
        
collection_title = '/mnt/data/factcheck/fever/data-cs/fever/title_as_id_fever.tsv'
# with open(collection, 'w') as fw:
#     for i, (d, dv) in enumerate(documents.items()):
#         fw.write(f"{d}\t{dv}\n")

In [46]:
!head -n 1 {collection}

0	Vietnamská kuchyně je důležitou částí vietnamské kultury . Jídlo představuje pro Vietnamce možnost strávit čas společně s rodinou a je hlavním prvkem oslav a svátků . Hlavní ingredience využívané ve vietnamské kuchyni v sobě odrážejí polohu země a místní klima . Rýže , která se pěstuje na vodních polích po celé zemi , tvoří součást každodenních jídel , a navíc se z ní vyrábějí různé druhy nudlí nebo koláčů . Kromě množství buddhistických vegetariánských pokrmů jsou vietnamská jídla kombinací rozmanitých druhů zeleniny , bylin a masa . Ingredience se připravují na mnoho způsobů , jako je vaření , dušení nebo smažení . Cílem kuchařů je zachování co nejčerstvější a přírodní chuti jídel . Vietnamská kuchyně je často považována za jednu z nejzdravějších na světě . Vietnamská kuchyně se řídí filozofií pěti elementů ( `` ngũ hành '' ) v pěti základních chutích : ostrá ( železo

In [47]:
!head -n 1 {collection_title}

Vietnamská kuchyně	Vietnamská kuchyně je důležitou částí vietnamské kultury . Jídlo představuje pro Vietnamce možnost strávit čas společně s rodinou a je hlavním prvkem oslav a svátků . Hlavní ingredience využívané ve vietnamské kuchyni v sobě odrážejí polohu země a místní klima . Rýže , která se pěstuje na vodních polích po celé zemi , tvoří součást každodenních jídel , a navíc se z ní vyrábějí různé druhy nudlí nebo koláčů . Kromě množství buddhistických vegetariánských pokrmů jsou vietnamská jídla kombinací rozmanitých druhů zeleniny , bylin a masa . Ingredience se připravují na mnoho způsobů , jako je vaření , dušení nebo smažení . Cílem kuchařů je zachování co nejčerstvější a přírodní chuti jídel . Vietnamská kuchyně je často považována za jednu z nejzdravějších na světě . Vietnamská kuchyně se řídí filozofií pěti elementů ( `` ngũ hành '' ) v pěti základních chutích

In [59]:
collection = {d: dv for i, (d, dv) in enumerate(documents.items())}

## Create training triples and dev queries

In [50]:
with jsonlines.open(dev) as fr:
    dev_claims = [c for c in fr]

with jsonlines.open(test) as fr:
    test_claims = [c for c in fr]
    
with jsonlines.open(train) as fr:
    train_claims = [c for c in fr]

In [51]:
print(f"train: {len(train_claims)}\tdev: {len(dev_claims)}")

train: 107330	dev: 9999


In [52]:
dev_claims[0]

{'id': 206088,
 'verifiable': 'NOT VERIFIABLE',
 'label': 'NOT ENOUGH INFO',
 'claim': 'Sociologie se zabývá vývojem politiky.',
 'evidence': [],
 'claim_en': 'Sociology is the study of politics development.'}

In [53]:
dev_queries_path = f'{ROOT}dev_queries.tsv'
with open(dev_queries_path, 'w') as fw:
    for i, c in enumerate(dev_claims):
        fw.write(f"{i}\t{c['claim']}\n")
        
test_queries_path = f'{ROOT}test_queries.tsv'
with open(test_queries_path, 'w') as fw:
    for i, c in enumerate(test_claims):
        fw.write(f"{i}\t{c['claim']}\n")

In [54]:
!head -n 2 {dev_queries_path}

0	Sociologie se zabývá vývojem politiky.
1	Sammy Cahn byl americký hudebník narozený v roce 1913.


In [55]:
!head -n 2 {test_queries_path}

0	Společnost Pixar financoval Steve Jobs.
1	Američtí bohové je román.


In [80]:
def get_random_doc(curr_doc_id, doc_ids):
    rand_doc_id = doc_ids[random.randint(0, len(doc_ids))]
    while curr_doc_id == rand_doc_id or rand_doc_id not in doc_ids:
        rand_doc_id = doc_ids[random.randint(0, len(doc_ids))]
    return rand_doc_id    

In [81]:
def get_tsv_file(claims, documents):
    tsv = []
    for claim in tqdm(claims):
        if claim['verifiable'] == 'VERIFIABLE':
            for evidence in claim['evidence']:
                pos_doc_id = evidence[0][2]  # doc containing evidence (positive)
                neg_doc_id = get_random_doc(pos_doc_id, list(documents.keys()))
                if pos_doc_id in documents:
                    tsv.append((f"{claim['claim'].strip()}\t"\
                                f"{documents[pos_doc_id].strip()}\t"\
                                f"{documents[neg_doc_id].strip()}\n"))
                else:
                    print(f"Document: {pos_doc_id} not found in the db.")
    return tsv

In [82]:
dev_tsv = get_tsv_file(dev_claims, collection)

  1%|          | 65/9999 [00:01<04:55, 33.57it/s]

Document: Molly Bernard not found in the db.


  4%|▍         | 414/9999 [00:09<05:21, 29.80it/s]

Document: Molly Bernard not found in the db.


  8%|▊         | 790/9999 [00:17<02:35, 59.24it/s]

...

 94%|█████████▍| 9436/9999 [03:15<00:09, 58.60it/s]

Document: Jak vycvičit draka (film, 2010) not found in the db.


 95%|█████████▍| 9469/9999 [03:15<00:09, 58.63it/s]

Document: Zemřít pro… not found in the db.


100%|██████████| 9999/9999 [03:27<00:00, 48.17it/s]


In [83]:
len(dev_tsv)

13794

In [84]:
dev_tsv[0]

"Sammy Cahn byl americký hudebník narozený v roce 1913.\tSammy Cahn , rodným jménem Samuel Cohen , ( 18. června 1913 – 15. ledna 1993 ) byl americký textař . Narodil se v New Yorku jako jediný syn židovských přistěhovalců z Haliče . Je autortem textů k muzikálu `` Walking Happy '' ( hudbu složil Jimmy Van Heusen ) , který byl roku 1966 představen na Broadwayi . V roce 1983 byl uveden do Songwriters Hall of Fame . Zemřel na srdeční selhání ve věku 79 let . Pohřben byl na Westwood Village Memorial Park Cemetery . Jeho synem byl kytarista Steve Khan .\tJosef Šich ( * 30. listopadu 1942 , Koválovice u Tištína ) je český římskokatolický kněz , emeritní kanovník olomoucké kapituly , amatérský fotograf . Po maturitě na jedenáctileté střední škole ve Zlíně odešel v roce 1960 studovat Cyrilometodějskou bohosloveckou fakultu v Litoměřicích , kde také 27. června 1965 přijal kněžské svěcení z rukou Františka Tomáška . Působil jako kap

In [71]:
train_tsv = get_tsv_file(train_claims, collection)

  2%|▏         | 2024/107330 [00:38<27:40, 63.43it/s]  

Document: Zemřít pro… not found in the db.


  3%|▎         | 3366/107330 [01:08<29:59, 57.77it/s]  

Document: Room 93 not found in the db.


  5%|▍         | 5087/107330 [01:46<29:00, 58.75it/s]  

...

 99%|█████████▉| 106384/107330 [36:33<00:17, 54.61it/s]

Document: Jak vycvičit draka (film, 2010) not found in the db.
Document: Jak vycvičit draka (film, 2010) not found in the db.
Document: Jak vycvičit draka (film, 2010) not found in the db.
Document: Jak vycvičit draka (film, 2010) not found in the db.
Document: Jak vycvičit draka (film, 2010) not found in the db.


100%|█████████▉| 106903/107330 [36:43<00:09, 45.40it/s]

Document: Studio Pierrot not found in the db.


100%|██████████| 107330/107330 [36:52<00:00, 48.51it/s]


In [85]:
def save(output_file, data):
    with open(output_file, 'w') as fw:
        for row in data:
            fw.write(row)

In [86]:
train_out = f'{ROOT}train.tsv'
dev_out = f'{ROOT}dev.tsv'

In [87]:
train_out

'/home/ryparmar/fever-cs-deepl/train.tsv'

In [88]:
# save(train_out, train_tsv)
save(dev_out, dev_tsv)

In [37]:
!ls /mnt/data/factcheck/fever/data-cs/fever-data/

dev.jsonl  dev.tsv  test.jsonl	train.jsonl  train.tsv


In [38]:
!head -n 1 /mnt/data/factcheck/fever/data-cs/fever-data/dev.tsv

Sammy Cahn byl americký hudebník narozený v roce 1913.	Sammy Cahn , rodným jménem Samuel Cohen , ( 18. června 1913 – 15. ledna 1993 ) byl americký textař . Narodil se v New Yorku jako jediný syn židovských přistěhovalců z Haliče . Je autortem textů k muzikálu `` Walking Happy '' ( hudbu složil Jimmy Van Heusen ) , který byl roku 1966 představen na Broadwayi . V roce 1983 byl uveden do Songwriters Hall of Fame . Zemřel na srdeční selhání ve věku 79 let . Pohřben byl na Westwood Village Memorial Park Cemetery . Jeho synem byl kytarista Steve Khan .	Pyssa nebo také Tyssa ( nebo ) je řeka v Republice Komi v Rusku . Je 164 km dlouhá . Povodí má rozlohu 1160 km2 .


In [39]:
!head -n 1 /mnt/data/factcheck/fever/data-cs/fever-data/train.tsv

Nelson Mandela zavedl bezplatnou zdravotní péči.	Nelson Rolihlahla Mandela ( 18. července 1918 – 5. prosince 2013 ) byl jihoafrický bojovník proti apartheidu , politik a prezident Jihoafrické republiky v letech 1994 až 1999 . Byl prvním černošským prezidentem zvoleným v prvních svobodných volbách v historii Jihoafrické republiky . Jeho vláda se soustředila na likvidaci dědictví apartheidu , zaměřovala se na institucionalizovaný rasismus , chudobu a sociální nerovnost , a zprostředkovala mezirasové usmíření v zemi . Politicky se hlásil k africkému nacionalismu a demokratickému socialismu . V letech 1991 až 1997 byl prezidentem Afrického národního kongresu . V mezinárodní politice byl generálním tajemníkem Hnutí nezúčastněných zemí v letech 1998 až 1999 . Narodil se jako příslušník královské rodiny národa Xhosů . Vystudoval právo na univerzitách v Alice a Johannesburgu . Po studiích se přidal k Africkému národnímu kongresu 